# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import nltk
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle


In [ ]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.iloc[:, 4:] 

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

pipeline.fit(X_train, Y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Report the f1 score, precision, and recall for each output category
for i, col in enumerate(Y.columns):
    print(f"Category: {col}\n{classification_report(Y_test[col], Y_pred[:, i])}")
    
# Using grid search to find better parameters   
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

# Fit the model with grid search
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Test the tuned model
Y_pred_tuned = cv.predict(X_test)

# Show the accuracy, precision, and recall of the tuned model
for i, col in enumerate(Y.columns):
    print(f"Tuned Category: {col}\n{classification_report(Y_test[col], Y_pred_tuned[:, i])}")


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.svm import SVC

svm_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

# Train the SVM model
svm_pipeline.fit(X_train, Y_train)

# Evaluate the model
Y_pred_svm = svm_pipeline.predict(X_test)
for i, col in enumerate(Y.columns):
    print(f"SVM Category: {col}\n{classification_report(Y_test[col], Y_pred_svm[:, i])}")

    from sklearn.ensemble import GradientBoostingClassifier

gb_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
])

# Train the Gradient Boosting model
gb_pipeline.fit(X_train, Y_train)

# Evaluate the model
Y_pred_gb = gb_pipeline.predict(X_test)
for i, col in enumerate(Y.columns):
    print(f"Gradient Boosting Category: {col}\n{classification_report(Y_test[col], Y_pred_gb[:, i])}")

    
# Feature engineering: add the length of messages as a feature
df['message_length'] = df['message'].apply(len)

# Update X and include the new feature
X = df[['message', 'message_length']]

# Update the pipeline to include the new feature
length_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('message_length', Pipeline([
            ('length', MessageLengthExtractor())  # Implement MessageLengthExtractor class
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# Train the model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
length_pipeline.fit(X_train, Y_train)

# Evaluate the model
Y_pred_length = length_pipeline.predict(X_test)
for i, col in enumerate(Y.columns):
    print(f"Category with Length Feature: {col}\n{classification_report(Y_test[col], Y_pred_length[:, i])}")
    

### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.